In [1]:
from qiskit import QuantumCircuit, execute
from math import pi

In [2]:
from qiskit.pulse import DriveChannel, GaussianSquare, ShiftPhase

def get_n_link(backend):
    configuration = backend.configuration()
    defaults = backend.defaults()
    n = configuration.num_qubits

    connected_pairs_cnot = configuration.coupling_map
    instruction_schedule_map = defaults.instruction_schedule_map

    def get_control_qubit(q1, q2):  # Control performs Z
        try:
            cx_sched = instruction_schedule_map.get("cx", qubits=(q1, q2))
        except:
            cx_sched = instruction_schedule_map.get("ecr", qubits=(q1, q2))
        supported = False
        for time, inst in cx_sched.instructions:
            if isinstance(inst.channel, DriveChannel) and not isinstance(inst, ShiftPhase):
                if isinstance(inst.pulse, GaussianSquare):
                    target = inst.channel.index
                    control = q1 if target == q2 else q2
                    supported = True
        if not supported:
            raise ValueError("This machine is not supported!")
        return control

    link = []
    for item in connected_pairs_cnot:
        if get_control_qubit(item[0], item[1]) == item[0]:
            link.append(item)

    return n, link

def make_circuit(backend,line,ins,params,t):
    n, link = get_n_link(backend)
    circ = QuantumCircuit(n)
    if line < n:
        if ins == 0:
            q = line
            circ.rz(-params[1], q)
            circ.rx(2 * params[0] * t, q)
            circ.rz(params[1], q)
        else:
            q = line
            lamb = 2 * params[0] * t
            circ.h(q)                   #Added to measure reliability
            circ.rz(lamb, q)
            circ.h(q)                   #Added to measure reliability
    else:
        # print(line)
        (q0, q1) = link[line - n]
        theta = 2 * params[0] * t
        if ins == 0:
            circ.rzx(theta, q0, q1)
        elif ins == 1:
            circ.rxx(theta, q0, q1)
        elif ins == 2:
            circ.ryy(theta, q0, q1)
        else:
            circ.h(q0)                   #Added to measure reliability
            circ.h(q1)                   #Added to measure reliability
            circ.rzz(theta, q0, q1)
            circ.h(q0)                   #Added to measure reliability
            circ.h(q1)                   #Added to measure reliability
    circ.measure_all()
    return circ

In [3]:
from qiskit import IBMQ

with open('../ibm_API_key','r') as file:
    token = file.readline()

provider = IBMQ.enable_account(token, hub='ibm-q-ncsu', group='nc-state', project='quantum-compiler')

#from qiskit_ibm_provider import IBMProvider
#IBMProvider.save_account(token,instance='ibm-q-ncsu/nc-state/quantum-compiler',overwrite=True)
#provider = IBMProvider()

C:\Users\Admin\AppData\Local\Temp\ipykernel_13128\335559639.py:6: DeprecationWarning: The package qiskit.providers.ibmq is being deprecated. Please see https://ibm.biz/provider_migration_guide to get instructions on how to migrate to qiskit-ibm-provider (https://github.com/Qiskit/qiskit-ibm-provider) and qiskit-ibm-runtime (https://github.com/Qiskit/qiskit-ibm-runtime).
  provider = IBMQ.enable_account(token, hub='ibm-q-ncsu', group='nc-state', project='quantum-compiler')
C:\Users\Admin\AppData\Local\Temp\ipykernel_13128\335559639.py:6: DeprecationWarning: The qiskit.IBMQ entrypoint and the qiskit-ibmq-provider package (accessible from 'qiskit.providers.ibmq`) are deprecated and will be removed in a future release. Instead you should use the qiskit-ibm-provider package which is accessible from 'qiskit_ibm_provider'. You can install it with 'pip install qiskit_ibm_provider'. Just replace 'qiskit.IBMQ' with 'qiskit_ibm_provider.IBMProvider'
  provider = IBMQ.enable_account(token, hub='ib

In [4]:
from qiskit_transpiler import get_pm
backend = provider.get_backend('ibmq_mumbai')
pm = get_pm(backend)

In [5]:
params = [pi/4,pi/2]
t = 1
n, links = get_n_link(backend)

In [6]:
circuits_0 = []
circuits_1 = []
circuits_link_0 = []
circuits_link_1 = []
circuits_link_2 = []
circuits_link_3 = []

for i in range(n):
    circuits_0.append(pm.run(make_circuit(backend,i,0,params,t)))
    circuits_1.append(pm.run(make_circuit(backend,i,1,params,t)))

for i in range(len(links)):
    circuits_link_0.append(pm.run(make_circuit(backend,n+i,0,params,t)))
    circuits_link_1.append(pm.run(make_circuit(backend,n+i,1,params,t)))
    circuits_link_2.append(pm.run(make_circuit(backend,n+i,2,params,t)))
    circuits_link_3.append(pm.run(make_circuit(backend,n+i,3,params,t)))

In [7]:
from qiskit import transpile as transpile_qiskit
transpiled_circuits_0 = transpile_qiskit(circuits_0,backend)
transpiled_circuits_1 = transpile_qiskit(circuits_1,backend)
transpiled_circuits_link_0 = transpile_qiskit(circuits_link_0,backend)
transpiled_circuits_link_1 = transpile_qiskit(circuits_link_1,backend)
transpiled_circuits_link_2 = transpile_qiskit(circuits_link_2,backend)
transpiled_circuits_link_3 = transpile_qiskit(circuits_link_3,backend)

In [8]:
from qiskit_aer.noise import NoiseModel
simulator = provider.get_backend("ibmq_qasm_simulator")
noise_model = NoiseModel.from_backend(backend).to_dict()
simulator.options.update_options(noise_model=noise_model)

In [9]:
job = execute(transpiled_circuits_0 * 5, shots=1024, backend=backend)
job = execute(transpiled_circuits_1 * 5, shots=1024, backend=backend)
job = execute(transpiled_circuits_link_0 * 5, shots=1024, backend=backend)
job = execute(transpiled_circuits_link_1 * 5, shots=1024, backend=backend)
job = execute(transpiled_circuits_link_2 * 5, shots=1024, backend=backend)
job = execute(transpiled_circuits_link_3 * 5, shots=1024, backend=backend)